#**02 - preprocesado**

**Miembros del grupo:**

* Cristian Camilo Serna Betancur, CC 1018351871, Ingeniería de Sistemas
* Diego Alonso Herrera Ramírez, CC 70908268, Ingeniería de Sistemas
* Sharid Samantha Madrid Ospina, CC 1001652997, Ingeniería de Sistemas


**Base de datos:**
* [hotel_booking](https://www.kaggle.com/datasets/saadharoon27/hotel-booking-dataset/data): Contiene información sobre reservas de hotel, que abarca detalles sobre los huéspedes, sus reservas y atributos del hotel

In [ ]:
import pandas as pd

# Se carga el conjunto de datos original
data = pd.read_csv("https://raw.githubusercontent.com/Udeaproject/IA-system/main/hotel_booking.csv", delimiter=",")

In [ ]:
# Contar la cantidad de datos por cada clase
class_counts = data['is_canceled'].value_counts()

# Obtener la cantidad mínima de muestras entre las clases
min_class_count = class_counts.min()

# Submuestrear ambas clases para igualar la cantidad mínima
canceled_0 = data[data['is_canceled'] == 0].sample(min_class_count)
canceled_1 = data[data['is_canceled'] == 1].sample(min_class_count)

# Combinar los datos submuestreados
balanced_data = pd.concat([canceled_0, canceled_1])

# Mezclar los datos para que no estén ordenados por clase
balanced_data = balanced_data.sample(frac=1, random_state=42)


In [ ]:
# Obtener el recuento de las clases
counts = balanced_data['is_canceled'].value_counts()

# Mostrar los recuentos
print(counts)

0    44224
1    44224
Name: is_canceled, dtype: int64


In [ ]:
# name, email, phone-number y credit-card son datos falsos y debido a
# se busca hacer predicciones sin contar con los datos personales de los
# usarios se ha decidido eliminar dichas columnas.
columns_to_discard = ['name', 'email', 'phone-number', 'credit_card']

# Se creea un nuevo DataFrame excluyendo las columnas anteriores.
db = balanced_data.drop(columns_to_discard, axis=1).copy()

# Se imprime la información del nuevo DataFrame
print("Información del nuevo conjunto de datos:")
print(db.info())

Información del nuevo conjunto de datos:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88448 entries, 38336 to 16682
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           88448 non-null  object 
 1   is_canceled                     88448 non-null  int64  
 2   lead_time                       88448 non-null  int64  
 3   arrival_date_year               88448 non-null  int64  
 4   arrival_date_month              88448 non-null  object 
 5   arrival_date_week_number        88448 non-null  int64  
 6   arrival_date_day_of_month       88448 non-null  int64  
 7   stays_in_weekend_nights         88448 non-null  int64  
 8   stays_in_week_nights            88448 non-null  int64  
 9   adults                          88448 non-null  int64  
 10  children                        88444 non-null  float64
 11  babies                          88448 non-null  

In [ ]:
# Debido a que en la columna "company" faltan el 94% de sus
# datos, se ha decidido eliminar la columna por  completo.
# Tambien se elimina la columna 'required_car_parking_spaces',
# debio a que es una columna constante que afecta negativamente
# la matriz de covarianza
columns_to_discard = ['company', 'required_car_parking_spaces']
db = db.drop(columns_to_discard, axis=1).copy()

# Se eliminan las filas donde 'agent', 'country', 'children' y 'adr' no tienen datos.
db = db.dropna(subset=['agent', 'country', 'children', 'adr'])

# Se convierte el tipo de la columna 'children' a int, actualmente esta en float64
db['children'] = db['children'].astype(int)


for column in db:
  missing_percentage = (db[column].isnull().sum() / len(db)) * 100
  print("\t{}: {:.2f}%".format(column, missing_percentage))

print(db.shape)
print(db.info())

	hotel: 0.00%
	is_canceled: 0.00%
	lead_time: 0.00%
	arrival_date_year: 0.00%
	arrival_date_month: 0.00%
	arrival_date_week_number: 0.00%
	arrival_date_day_of_month: 0.00%
	stays_in_weekend_nights: 0.00%
	stays_in_week_nights: 0.00%
	adults: 0.00%
	children: 0.00%
	babies: 0.00%
	meal: 0.00%
	country: 0.00%
	market_segment: 0.00%
	distribution_channel: 0.00%
	is_repeated_guest: 0.00%
	previous_cancellations: 0.00%
	previous_bookings_not_canceled: 0.00%
	reserved_room_type: 0.00%
	assigned_room_type: 0.00%
	booking_changes: 0.00%
	deposit_type: 0.00%
	agent: 0.00%
	days_in_waiting_list: 0.00%
	customer_type: 0.00%
	adr: 0.00%
	total_of_special_requests: 0.00%
	reservation_status: 0.00%
	reservation_status_date: 0.00%
(77034, 30)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 77034 entries, 38336 to 16682
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                 

In [ ]:
categorical_columns = [
    'hotel',
    'meal',
    'arrival_date_month',
    'country',
    'market_segment',
    'distribution_channel',
    'reserved_room_type',
    'assigned_room_type',
    'deposit_type',
    'customer_type',
    'reservation_status'
]

In [ ]:
# Se convierten los datos categóricos en datos numéricos
for column in categorical_columns:
  ordinal_mapping = {k: i + 1 for i, k in enumerate(db[column].unique())}
  print("Columna categoria: {}: {}".format(column, ordinal_mapping))
  db[column] = db[column].map(ordinal_mapping)
print(db.info())

Columna categoria: hotel: {'Resort Hotel': 1, 'City Hotel': 2}
Columna categoria: meal: {'BB': 1, 'HB': 2, 'SC': 3, 'Undefined': 4, 'FB': 5}
Columna categoria: arrival_date_month: {'July': 1, 'April': 2, 'May': 3, 'February': 4, 'March': 5, 'August': 6, 'September': 7, 'December': 8, 'November': 9, 'October': 10, 'January': 11, 'June': 12}
Columna categoria: country: {'PRT': 1, 'IRL': 2, 'ESP': 3, 'NLD': 4, 'FRA': 5, 'AGO': 6, 'CHN': 7, 'POL': 8, 'USA': 9, 'GBR': 10, 'AUT': 11, 'DEU': 12, 'IRN': 13, 'LVA': 14, 'AUS': 15, 'CN': 16, 'CHE': 17, 'NOR': 18, 'FIN': 19, 'ITA': 20, 'BEL': 21, 'CPV': 22, 'ROU': 23, 'COL': 24, 'IND': 25, 'BRA': 26, 'ISR': 27, 'RUS': 28, 'SRB': 29, 'SWE': 30, 'GNB': 31, 'GRC': 32, 'DNK': 33, 'BGR': 34, 'CUB': 35, 'URY': 36, 'ZAF': 37, 'TUR': 38, 'THA': 39, 'CYP': 40, 'MEX': 41, 'DOM': 42, 'PHL': 43, 'NZL': 44, 'CHL': 45, 'MAR': 46, 'KOR': 47, 'MYS': 48, 'CZE': 49, 'BLR': 50, 'JPN': 51, 'HUN': 52, 'LTU': 53, 'DZA': 54, 'PER': 55, 'NGA': 56, 'TUN': 57, 'KWT': 58, '

In [ ]:
# Se separa reservation_status_date en 3 columnas: day, year y month
column = "reservation_status_date"
db[column] = pd.to_datetime(db[column])
db['reservation_status_date_day'] = db[column].dt.day
db['reservation_status_date_year'] = db[column].dt.year
db['reservation_status_date_month'] = db[column].dt.month
db = db.drop([column], axis=1).copy()

print(db.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77034 entries, 38336 to 16682
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           77034 non-null  int64  
 1   is_canceled                     77034 non-null  int64  
 2   lead_time                       77034 non-null  int64  
 3   arrival_date_year               77034 non-null  int64  
 4   arrival_date_month              77034 non-null  int64  
 5   arrival_date_week_number        77034 non-null  int64  
 6   arrival_date_day_of_month       77034 non-null  int64  
 7   stays_in_weekend_nights         77034 non-null  int64  
 8   stays_in_week_nights            77034 non-null  int64  
 9   adults                          77034 non-null  int64  
 10  children                        77034 non-null  int64  
 11  babies                          77034 non-null  int64  
 12  meal                        

In [ ]:
db

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,days_in_waiting_list,customer_type,adr,total_of_special_requests,reservation_status,reservation_status_date_day,reservation_status_date_year,reservation_status_date_month
38336,1,0,142,2017,1,28,10,1,4,2,...,1,240.0,0,1,154.00,1,1,15,2017,7
23099,1,0,3,2016,2,16,14,0,1,2,...,1,250.0,0,2,125.00,0,1,15,2016,4
68822,2,1,25,2017,3,21,21,2,0,1,...,1,9.0,0,1,160.00,1,2,26,2017,4
12656,1,1,163,2017,1,28,10,1,5,2,...,1,240.0,0,1,182.00,0,2,30,2017,1
24813,1,0,143,2016,3,22,28,2,5,2,...,1,240.0,0,1,76.19,1,1,4,2016,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37750,1,0,222,2017,12,25,23,2,3,2,...,1,241.0,0,1,76.23,1,1,28,2017,6
59741,2,1,166,2016,9,45,1,0,3,1,...,2,236.0,0,1,110.00,0,2,13,2016,7
10843,1,1,95,2017,2,14,8,2,3,3,...,1,240.0,0,1,150.00,0,2,19,2017,1
94787,2,0,0,2016,6,33,9,0,1,2,...,1,9.0,0,1,158.00,0,1,10,2016,8


In [ ]:
# Se imprime el valor minimo, maximo y medio de cada columna
for column in db.columns:
  min_value = db[column].min()
  max_value = db[column].max()
  mean_value = db[column].mean()
  print("{}:".format(column))
  print("\tmin:", min_value)
  print("\tmax:", max_value)
  print("\tmean: {:.2f}".format(mean_value))

hotel:
	min: 1
	max: 2
	mean: 1.71
is_canceled:
	min: 0
	max: 1
	mean: 0.52
lead_time:
	min: 0
	max: 629
	mean: 120.08
arrival_date_year:
	min: 2015
	max: 2017
	mean: 2016.16
arrival_date_month:
	min: 1
	max: 12
	mean: 6.08
arrival_date_week_number:
	min: 1
	max: 53
	mean: 27.31
arrival_date_day_of_month:
	min: 1
	max: 31
	mean: 15.78
stays_in_weekend_nights:
	min: 0
	max: 16
	mean: 0.97
stays_in_week_nights:
	min: 0
	max: 41
	mean: 2.59
adults:
	min: 0
	max: 26
	mean: 1.90
children:
	min: 0
	max: 10
	mean: 0.11
babies:
	min: 0
	max: 9
	mean: 0.01
meal:
	min: 1
	max: 5
	mean: 1.38
country:
	min: 1
	max: 160
	mean: 8.67
market_segment:
	min: 1
	max: 7
	mean: 2.11
distribution_channel:
	min: 1
	max: 4
	mean: 1.09
is_repeated_guest:
	min: 0
	max: 1
	mean: 0.01
previous_cancellations:
	min: 0
	max: 26
	mean: 0.10
previous_bookings_not_canceled:
	min: 0
	max: 25
	mean: 0.02
reserved_room_type:
	min: 1
	max: 8
	mean: 1.69
assigned_room_type:
	min: 1
	max: 10
	mean: 1.91
booking_changes:
	min

In [ ]:
# Se guarda el nuevo DataFrame en un archivo CSV
db.to_csv('preprocessed_hotel_booking.csv', index=False)